In [2]:
import json
import os
import re 

In [3]:
def remove_brackets(data):
    bracket_items = []
    for index in range(len(data)):
        brackets = re.findall('\[.*?\]', data[index]["text"])
        if len(brackets) != 0:
            bracket_items += brackets
    bracket_items = set(bracket_items)

    for index in range(len(data)):
        for bracket_item in bracket_items:
            if bracket_item in data[index]["text"]:
                data[index]["text"] = data[index]["text"].replace(bracket_item, "")
        data[index]["text"] = data[index]["text"].strip()
        
    return data

def clean_generated_data(data):
    data = remove_brackets(data)

    for index in range(len(data)):
        example = dict(data[index])
        if "generated_raw" in example:
            del example["generated_raw"]        
        if "id" not in example:
            example["id"] = "generated"
        data[index] = example

    for index in range(len(data)):
        if ": " in data[index]["text"]:
            data[index]["text"] = data[index]["text"].replace(": ", "")
        data[index]["text"] = data[index]["text"].strip()

    return data

plutchik = ["anger", "fear", "sadness", "disgust", "anticipation", "trust", "joy"]

def get_emotion_counts(data):

    emotion_counts = {}
    for emotion in plutchik:
        emotion_counts[emotion] = 0
    for example in data:
        emotion_counts[example["choice"]] += 1
    
    return emotion_counts

def get_duplicates(data):
    duplicates = []
    for i in range(len(data)):
        for j in range(i+1, len(data)):
            if data[i] == data[j]:
                duplicates.append((i,j))    

def remove_duplicates(data):
    return [dict(t) for t in {tuple(d.items()) for d in data}]

In [9]:
full_data = json.load(open("../data/english/full_data.json", "r"))

new_examples = []
for folder in os.listdir("../data/english/"):
    if "process_id" in folder:
        new_examples += json.load(open(os.path.join("../data/english/", folder), "r"))

cleaned_new_examples = clean_generated_data(new_examples)
full_data += cleaned_new_examples
full_data = clean_generated_data(full_data)
print(len(full_data))
full_data = remove_duplicates(full_data)
print(len(full_data))

5226
4946


In [10]:
get_emotion_counts(full_data)

{'anger': 708,
 'fear': 704,
 'sadness': 702,
 'disgust': 707,
 'anticipation': 709,
 'trust': 707,
 'joy': 709}

In [7]:
json.dump(full_data, open("../data/english/full_data.json", "w"), indent=4)